### Importing Libraries
Import the required libraries for data analysis.

In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
class GraphDataset(Dataset):
    FILE_PATH = ''

    def __init__(self, user_id=None, label_list=None, num_samples=None, secs=None):
        self.user_id = user_id
        self.label_list = label_list
        self.num_samples = num_samples
        self.secs = secs

        self.data, self.labels = self.load_data(user_id, label_list, secs)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.Tensor(self.data[idx]), self.labels[idx]

    def load_data(self, user_id, label_list, secs):
        data = list()
        labels = list()

        for i in range(len(label_list)):
            for j in range(1, num_samples + 1):
                try:
                    adjMatrix = scipy.io.loadmat(
                        self.FILE_PATH + str(user_id) +
                        '_label' + str(label_list[i]) +
                        '_item' + str(j) + '.mat')

                except Exception as e:
                    print(f"Load fail: {e}")
                    continue

                if self.secs == 1:
                    tmpAdjMatrix = adjMatrix['fullMatrix'][np.newaxis, :, :]
                else:
                    tmpAdjMatrix = list()
                    for k in range(self.secs):
                        tmpAdjMatrix.append(adjMatrix['fullMatrix'][:, :, k])

                data.append(np.array(tmpAdjMatrix))
                labels.append(label_list[i])

        if not data or not labels:
            raise RuntimeError("No data or labels loaded.")

        data = torch.tensor(np.stack(data), dtype=torch.float32)

        label_dict = {label: index for index, label in enumerate(self.label_list)}
        new_label_list = [label_dict[label] for label in labels]
        labels = torch.tensor(new_label_list, dtype=torch.long)

        return data, labels

### Loading Data
Load the full dataset.

In [ ]:
user_id = 1
label_list = [770, 772]
num_samples = 72
secs = 1

In [ ]:
full_dataset = GraphDataset(user_id=user_id, label_list=label_list, num_samples=num_samples, secs=secs)

Create the train and test set.

In [ ]:
train_size = int(0.75 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [ ]:
batch_size = 6

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

## Classification

In [ ]:
class CNNGeneral(nn.Module):
    def __init__(self, label_list):
        super(CNNGeneral, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 2 * 2, 120)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(label_list))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
class CNNGeneralImproved(nn.Module):
    def __init__(self, label_list):
        super(CNNGeneralImproved, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.norm1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.norm2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 2 * 2, 64)
        self.norm3 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, len(label_list))
        self.dropout2 = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pool(F.relu(self.norm1(self.conv1(x))))
        x = self.pool(F.relu(self.norm2(self.conv2(x))))
        x = torch.flatten(x, 1)
        x = F.relu(self.norm3(self.fc1(x)))
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        return x

In [ ]:
class CNNDepression(nn.Module):
    def __init__(self, label_list):
        super(CNNDepression, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(16 * 2 * 2, len(label_list))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        x = self.dropout1(x)
        x = F.softmax(x, dim=0)

        return x

In [ ]:
class CNNMIAged(nn.Module):
    def __init__(self, label_list):
        super(CNNMIAged, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.norm1 = nn.BatchNorm2d(6)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.norm2 = nn.BatchNorm2d(16)
        self.dropout1 = nn.Dropout(0.5)
        self.pool1 = nn.AvgPool2d(2, 2)

        self.conv3 = nn.Conv2d(16, 16, kernel_size=5)  # Corregido el número de canales de entrada
        self.norm3 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout(0.5)
        self.pool2 = nn.AvgPool2d(2, 2)

        self.dropout3 = nn.Dropout(0.5)

        self.fc1 = nn.Linear(16, len(label_list))

    def forward(self, x):
        x = F.elu(self.norm1(self.conv1(x)))

        x = F.elu(self.norm2(self.conv2(x)))
        x = self.dropout1(x)
        x = self.pool1(x)

        x = F.elu(self.norm3(self.conv3(x)))
        x = self.dropout2(x)
        x = self.pool2(x)

        x = torch.flatten(x, 1)

        x = self.dropout3(x)

        x = self.fc1(x)

        return x

In [ ]:
net = CNNMIAged(label_list)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 15 == 0:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 15:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.042
[1,    16] loss: 0.688
[2,     1] loss: 0.034
[2,    16] loss: 0.458
[3,     1] loss: 0.052
[3,    16] loss: 0.439
[4,     1] loss: 0.012
[4,    16] loss: 0.431
[5,     1] loss: 0.021
[5,    16] loss: 0.328
[6,     1] loss: 0.013
[6,    16] loss: 0.299
[7,     1] loss: 0.013
[7,    16] loss: 0.271
[8,     1] loss: 0.009
[8,    16] loss: 0.332
[9,     1] loss: 0.040
[9,    16] loss: 0.284
[10,     1] loss: 0.012
[10,    16] loss: 0.288
[11,     1] loss: 0.036
[11,    16] loss: 0.271
[12,     1] loss: 0.010
[12,    16] loss: 0.286
[13,     1] loss: 0.008
[13,    16] loss: 0.220
[14,     1] loss: 0.018
[14,    16] loss: 0.246
[15,     1] loss: 0.014
[15,    16] loss: 0.261
[16,     1] loss: 0.009
[16,    16] loss: 0.231
[17,     1] loss: 0.029
[17,    16] loss: 0.240
[18,     1] loss: 0.010
[18,    16] loss: 0.301
[19,     1] loss: 0.005
[19,    16] loss: 0.247
[20,     1] loss: 0.011
[20,    16] loss: 0.231
[21,     1] loss: 0.009
[21,    16] loss: 0.220
[22,     1

[170,    16] loss: 0.030
[171,     1] loss: 0.001
[171,    16] loss: 0.044
[172,     1] loss: 0.001
[172,    16] loss: 0.066
[173,     1] loss: 0.001
[173,    16] loss: 0.026
[174,     1] loss: 0.001
[174,    16] loss: 0.024
[175,     1] loss: 0.000
[175,    16] loss: 0.016
[176,     1] loss: 0.002
[176,    16] loss: 0.015
[177,     1] loss: 0.004
[177,    16] loss: 0.024
[178,     1] loss: 0.001
[178,    16] loss: 0.018
[179,     1] loss: 0.008
[179,    16] loss: 0.068
[180,     1] loss: 0.002
[180,    16] loss: 0.033
[181,     1] loss: 0.000
[181,    16] loss: 0.040
[182,     1] loss: 0.000
[182,    16] loss: 0.011
[183,     1] loss: 0.000
[183,    16] loss: 0.043
[184,     1] loss: 0.000
[184,    16] loss: 0.032
[185,     1] loss: 0.003
[185,    16] loss: 0.015
[186,     1] loss: 0.003
[186,    16] loss: 0.032
[187,     1] loss: 0.001
[187,    16] loss: 0.027
[188,     1] loss: 0.000
[188,    16] loss: 0.014
[189,     1] loss: 0.002
[189,    16] loss: 0.044
[190,     1] loss: 0.001


In [ ]:
# prepare to count predictions for each class
classes = ["0", "1", "2", "3"]

correct = 0
total = 0

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predictions == labels).sum().item()

        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print(f'Accuracy of the network on the test: {100 * correct // total} %')

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 0

    if total_pred[classname] != 0:
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy of the network on the test: 94 %
Accuracy for class: 0     is 93.8 %
Accuracy for class: 1     is 95.0 %
